In [ ]:
import sys
sys.path.append("src")

import generator

from pathlib import Path

from PIL import Image
import cv2
from skimage import filters

import pytesseract
from pytesseract import Output
import numpy as np
from tqdm import tqdm
import json

In [ ]:
config = {
"digits": "data/digits/semeion.data",
"circles": "data/circles/full_numpy_bitmap_circle.npy",
"arrows": "data/arrow_foreground",
"lines": "data/lines/full_numpy_bitmap_line.npy",
"words": "data/word_foreground",
"sentences": "data/sentence_foreground"
}


gen = generator.generator(config)

training_path = Path('data/form_background/data/imgs/')
training_background = [f for f in training_path.glob('**/*.png')]

In [ ]:
len(training_background)

In [ ]:
N_SAMPLES = 1000
OUTPUT_DIR = "data/train/"

In [ ]:
for index in tqdm(range(N_SAMPLES), total = N_SAMPLES):

    # choose random background
    random_choice = np.random.randint(len(training_background))
    path = training_background[random_choice]

    # prepare background
    background = cv2.imread(path.as_posix(), cv2.IMREAD_COLOR)
    background = cv2.resize(background, (2048, 2048), cv2.INTER_AREA)
    background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
    th = filters.threshold_sauvola(background, window_size=15, k=0.2)
    background = background > th
    background = background.astype('uint8')
    background_w, background_h = background.shape

    foreground = np.ones((background_w, background_h))
    foreground = foreground.astype('float64')  
    background = background.astype('float64')


    gen.set_foreground(foreground)

    background[background==1] = 2
    background[background==0] = 1
    background[background==2] = 0

    with open(training_path.parent / 'jsons' / (path.stem + '.json'), 'r') as f:
        data = json.load(f)

    for i in data:
        if i == 'Widget':
            for j in data[i]:
                if int(j['w']) > 30 and int(j['h']) > 30:
                    found = gen.put_text_on_location(background, \
                                                   (int(j['x']), int(j['y']), int(j['w']), int(j['h'])))
    foreground = gen.foreground.copy()

    foreground[foreground==1] = 2
    foreground[foreground==0] = 1
    foreground[foreground==2] = 0

    # create training masks
    comb = background + foreground

    comb[comb==1] = 2
    comb[comb==0] = 1
    comb[comb==2] = 0


    # reset foreground
    gen.reset_foreground()


    filename = path.stem
    dest_filename_gt = OUTPUT_DIR + filename + '_' + str(index) + "_" +  '_FORM' + '_gt.png'
    cv2.imwrite(dest_filename_gt, foreground.astype('uint8') * 255)

    # save TR
    dest_filename_tr = OUTPUT_DIR + filename + '_' + str(index) + "_" + '_FORM' +'_tr.png'
    cv2.imwrite(dest_filename_tr, comb.astype('uint8') * 255)

In [ ]:
N_SAMPLES = 100
OUTPUT_DIR = "data/val/"

In [ ]:
for index in tqdm(range(N_SAMPLES), total = N_SAMPLES):

    # choose random background
    random_choice = np.random.randint(len(training_background))
    path = training_background[random_choice]

    # prepare background
    background = cv2.imread(path.as_posix(), cv2.IMREAD_COLOR)
    background = cv2.resize(background, (2048, 2048), cv2.INTER_AREA)
    background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
    th = filters.threshold_sauvola(background, window_size=15, k=0.2)
    background = background > th
    background = background.astype('uint8')
    background_w, background_h = background.shape

    foreground = np.ones((background_w, background_h))
    foreground = foreground.astype('float64')  
    background = background.astype('float64')


    gen.set_foreground(foreground)

    background[background==1] = 2
    background[background==0] = 1
    background[background==2] = 0

    with open(training_path.parent / 'jsons' / (path.stem + '.json'), 'r') as f:
        data = json.load(f)

    for i in data:
        if i == 'Widget':
            for j in data[i]:
                if int(j['w']) > 30 and int(j['h']) > 30:
                    found = gen.put_text_on_location(background, \
                                                   (int(j['x']), int(j['y']), int(j['w']), int(j['h'])))
    foreground = gen.foreground.copy()

    foreground[foreground==1] = 2
    foreground[foreground==0] = 1
    foreground[foreground==2] = 0

    # create training masks
    comb = background + foreground

    comb[comb==1] = 2
    comb[comb==0] = 1
    comb[comb==2] = 0


    # reset foreground
    gen.reset_foreground()


    filename = path.stem
    dest_filename_gt = OUTPUT_DIR + filename + '_' + str(index) + "_" +  '_FORM' + '_gt.png'
    cv2.imwrite(dest_filename_gt, foreground.astype('uint8') * 255)

        
    dest_filename_gt = OUTPUT_DIR + filename + '_' + str(index) + "_" +   '_ARCHIVE' + '_orig.png'
    cv2.imwrite(dest_filename_gt, background.astype('uint8') * 255)


    # save TR
    dest_filename_tr = OUTPUT_DIR + filename + '_' + str(index) + "_" + '_FORM' +'_tr.png'
    cv2.imwrite(dest_filename_tr, comb.astype('uint8') * 255)